#### with 문????

파이썬의 with 문은 코드를 특별한 컨텍스트에서 실행함을 나타낸다.
예를들면 어떤 코드를 lock을 사용하여 잠금이 설정되어 있는 동안만 실행하고 싶다면?

In [4]:
from threading import Lock

lock = Lock()
with lock:
    print('Lock is held')

Lock is held


위의 코드는 Lock 클래스가 with 문을 제대로 지원하는 덕분이다. 위의 코드를 try / catch 문을 사용해서 표현하면

In [5]:
lock.acquire()
try:
    print('Lock is held')
finally:
    lock.release()

Lock is held


특정 코드 영역에 저 많은 디버깅 로그를 넣고 싶다고 가정하자.

In [8]:
import logging

def my_function():
    logging.debug('Some debug data')
    logging.error('Error log here')
    logging.debug('More debug data')

my_function()

ERROR:root:Error log here


이 프로그램의 기본 로그 수준은 WARNING 이기 때문에. debug 메시지를 출력되지 않고 error 메시지만 출력한다.

컨텍스트 매니저를 정의해서 이 함수의 로그 수준을 임시로 높일 수 있다.
with 블록에서 코드를 실행하기 전에 로그 심각성 수준을 높이고 실행 후에는 다시 낮춘다.

In [13]:
from contextlib import contextmanager

@contextmanager
def debug_logging(level):
    logger = logging.getLogger()
    old_level = logger.getEffectiveLevel()
    logger.setLevel(level)
    try:
        yield
    finally:
        logger.setLevel(old_level)

yield 표현식은 with 블록의 내용이 실행되는 지점이다.

In [14]:
with debug_logging(logging.DEBUG):
    print('Inside:')
    my_function()
print('After:')
my_function()

DEBUG:root:Some debug data
ERROR:root:Error log here
DEBUG:root:More debug data
ERROR:root:Error log here


yield
Inside:
After:


이제 같은 호깅 함수를 debug_logging 컨텍스트 에서 호출하면, 디버그 메시지까지 모두 출력된다. 같은 함수를 with 블록 외부에서 실행하면 디버깅 메시지가 출력되지 않는다.

### with 타깃 사용하기 (as)

with 문에 전달되는 컨텍스트 매니저에서 객체를 반환할 수도 있다. 이 객체는 as 부분에 있는 지역 변수에 할당된다. 이 기능을 이용하면 with 블록 안에 있는 코드에서 직접 컨텍스트와 상호작용할 수 있다.

In [20]:
@contextmanager
def my_open(file, mode, close_handler):
    try:
        f = open(file, mode)
        yield f
    finally:
        close_handler()
        f.close()

In [21]:
def close_handler():
    print('file close')

with my_open('abc', 'r', close_handler) as f:
    b = f.read()

file close


file 핸들러가 닫히기 전에 어떤 함수를 실행하고 싶을때, 위와 같이 컨텍스트 매니저를 이용하면 가능하다

In [22]:
@contextmanager
def log_level(level, name):
    logger = logging.getLogger(name)
    old_level = logger.getEffectiveLevel()
    logger.setLevel(level)
    try:
        yield logger
    finally:
        logger.setLevel(old_level)

기본 logger의 심각성 수준은 WARNING 이므로 loggin 모듈을 직적 사용하면 아무것도 출력되지 않지만, contextmanager를 이용해 할당받은 logger는 디버그 메시지도 출력이 된다. 

In [23]:
with log_level(logging.DEBUG, 'my-log') as logger:
    logger.debug('This is my message')
    logging.debug('This will not print')

DEBUG:my-log:This is my message


with 문이 종료한 후에 my-log 로거를 이용해 로그 메시지를 출력해보면, 로깅 심각성 수준이 WARNING 으로 돌아가서 디버그메시지는 출력되지 않는다.

In [24]:
logger = logging.getLogger('my-log')
logger.debug('Debug will not print')
logger.error('Error will print')

ERROR:my-log:Error will print
